In [1]:
'''Instalar Sklearn para usar mean_squared_error '''
%pip install sklearn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,mean_absolute_percentage_error, r2_score,mean_absolute_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
from datetime import datetime
import matplotlib.pyplot as plt

In [3]:
def Bias(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_true = y_true.reshape(len(y_true),1)
    y_pred = y_pred.reshape(len(y_pred),1)   
    diff = (y_pred-y_true)
    bias = diff.sum()*100/y_true.sum()
    print('BIAS = ', bias)

def trans_fecha(data):
    if 'Date' in data.columns:
        data.rename(columns={'Date':'fecha'}, inplace=True)
    data['fecha'] = pd.to_datetime(data['fecha'])
    return data

def mes(data, mes):
    data = data[data['fecha'].dt.month == mes]
    data.reset_index(drop=True, inplace=True)
    return data

def media_mensual(data):
    data = data.drop('fecha',axis=1)
    data = data.reindex(columns = data.columns.sort_values(ascending=True))
    data = data.mean()
    data.sort_index(ascending=True, inplace=True)
    return data

# PARA PRECIPITACION

In [4]:
'''Datos Mensuales Sin Correcion'''
ppobs = pd.read_csv('../datos/estaciones_climatol/pre-m_1981-2020_series.csv')   #Corregidos climatol
ppchirp = pd.read_csv('../datos/por_variables/chirps_mensual_pp.csv')

In [5]:
'''Cálculo de correciones BIAS'''
ppobs = trans_fecha(ppobs)
ppchirp = trans_fecha(ppchirp)

ppchirp_correg = pd.DataFrame()

for m in range(1,13):
    obs_m = mes(ppobs, m)
    media_mes_obs = media_mensual(obs_m)
    
    chirp_m = mes(ppchirp, m)
    media_mes_chirp = media_mensual(chirp_m)
    
    mensual_correg = pd.DataFrame(chirp_m['fecha'], columns=chirp_m.columns)

    res = pd.Series(index = media_mes_chirp.index, dtype='float64')
    for indice in media_mes_chirp.index:
        if indice in media_mes_obs:
            res[indice] = media_mes_obs[indice] / media_mes_chirp[indice]
        else:
            res[indice] = 1.0
        
    for estacion in media_mes_chirp.index:
        mensual_correg.loc[:,estacion] = chirp_m.loc[:,estacion]*res[estacion]
    ppchirp_correg = pd.concat([ppchirp_correg, mensual_correg], axis = 0)
ppchirp_correg.reset_index(drop=True, inplace=True)
ppchirp_correg = ppchirp_correg.sort_values('fecha', ascending = True)
ppchirp_correg.reset_index(drop=True, inplace=True)
ppchirp_correg.to_csv(r'../datos/por_variables/chirps_mensual_pp_bc.csv', index=False)


# PARA TEMPERATURA MAXIMA

In [6]:
txobs = pd.read_csv('../datos/estaciones_climatol/tmax-m_1981-2020_series.csv')
txnasa = pd.read_csv('../datos/por_variables/nasa_mensual_tmax.csv')

In [7]:
'''Cálculo de correciones BIAS'''
txobs = trans_fecha(txobs)
txnasa = trans_fecha(txnasa)

txnasa_correg = pd.DataFrame()

for m in range(1,13):
    obs_m = mes(txobs, m)
    media_mes_obs = media_mensual(obs_m)
    
    nasa_m = mes(txnasa, m)
    media_mes_nasa = media_mensual(nasa_m)
    
    mensual_correg = pd.DataFrame(nasa_m['fecha'], columns=nasa_m.columns)

    res = pd.Series(index = media_mes_nasa.index, dtype='float64')
    for indice in media_mes_nasa.index:
        if indice in media_mes_obs:
            res[indice] = media_mes_obs[indice] / media_mes_nasa[indice]
        else:
            res[indice] = 1.0
        
    for estacion in media_mes_nasa.index:
        mensual_correg.loc[:,estacion] = nasa_m.loc[:,estacion]*res[estacion]
    txnasa_correg = pd.concat([txnasa_correg, mensual_correg], axis = 0)
txnasa_correg.reset_index(drop=True, inplace=True)
txnasa_correg = txnasa_correg.sort_values('fecha', ascending = True)
txnasa_correg.reset_index(drop=True, inplace=True)
txnasa_correg.to_csv(r'../datos/por_variables/nasa_mensual_tmax_bc.csv', index=False)

# PARA TEMPERATURA MINIMA

In [8]:
tnobs = pd.read_csv('../datos/estaciones_climatol/tmin-m_1981-2020_series.csv')
tnnasa = pd.read_csv('../datos/por_variables/nasa_mensual_tmin.csv')

In [9]:
'''Cálculo de correciones BIAS'''
tnobs = trans_fecha(tnobs)
tnnasa = trans_fecha(tnnasa)

tnnasa_correg = pd.DataFrame()

for m in range(1,13):
    obs_m = mes(tnobs, m)
    media_mes_obs = media_mensual(obs_m)
    
    nasa_m = mes(tnnasa, m)
    media_mes_nasa = media_mensual(nasa_m)
    
    mensual_correg = pd.DataFrame(nasa_m['fecha'], columns=nasa_m.columns)

    res = pd.Series(index = media_mes_nasa.index, dtype='float64')
    for indice in media_mes_nasa.index:
        if indice in media_mes_obs:
            res[indice] = media_mes_obs[indice] / media_mes_nasa[indice]
        else:
            res[indice] = 1.0
        
    for estacion in media_mes_nasa.index:
        mensual_correg.loc[:,estacion] = nasa_m.loc[:,estacion]*res[estacion]
    tnnasa_correg = pd.concat([tnnasa_correg, mensual_correg], axis = 0)
tnnasa_correg.reset_index(drop=True, inplace=True)
tnnasa_correg = tnnasa_correg.sort_values('fecha', ascending = True)
tnnasa_correg.reset_index(drop=True, inplace=True)
tnnasa_correg.to_csv(r'../datos/por_variables/nasa_mensual_tmin_bc.csv', index=False)
